# GBP SONIA Interest Rate Swap Pricing

This notebook reconstructs the forward and discount curves, projects cash flows for both legs of a SONIA-referenced GBP interest rate swap, and computes the mark-to-market value as of the chosen valuation date.


### Imports and Path Setup
Load Python packages and ensure the project `src` modules are reachable from the notebook.


In [1]:
from datetime import date
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Ensure the src package is importable
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.market_data import load_ois_quotes, load_forward_quotes
from src.curves import ZeroCurve, CurvePoint
from src.swap_pricing import SwapDefinition, SwapPricer


### Load Synthetic Market Data and Build Curves
Read SONIA OIS and swap quotes, then construct discount and forward zero curves and their dataframes.


In [2]:
ois_df = load_ois_quotes()
forward_df = load_forward_quotes()

discount_curve = ZeroCurve.from_par_swap_dataframe(ois_df, name="SONIA OIS Discount", payment_frequency=4)
forward_curve = ZeroCurve.from_par_swap_dataframe(forward_df, name="SONIA Forward", payment_frequency=4)

discount_curve_df = discount_curve.as_dataframe()
forward_curve_df = forward_curve.as_dataframe().rename(columns={"zero_rate": "forward_zero_rate", "discount_factor": "forward_discount"})

ois_df


,instrument_type,tenor_years,rate
0,OIS_MARKET,1,0.0305
1,OIS_MARKET,2,0.0310
2,OIS_MARKET,3,0.0318
3,OIS_MARKET,4,0.0326
4,OIS_MARKET,5,0.0335
5,OIS_MARKET,6,0.0340
6,OIS_MARKET,7,0.0345
7,OIS_MARKET,8,0.0350
8,OIS_MARKET,9,0.0355
9,OIS_MARKET,10,0.0360


### Derive Implied Forward Rates from Discount Factors
Compute the annualized forward rate from time 0 to each tenor using the OIS discount factors.


In [3]:
discount_curve_df["implied_forward_rate_0_T"] = discount_curve_df["discount_factor"].shift(fill_value=1.0) / discount_curve_df["discount_factor"]
discount_curve_df["implied_forward_rate_0_T"] = (
    discount_curve_df["implied_forward_rate_0_T"] - 1
) / discount_curve_df["tenor_years"].replace(0, np.nan)
discount_curve_df


,tenor_years,zero_rate,discount_factor,implied_forward_rate_0_T
0,0.25,0.030384,0.992433,0.030500
1,0.50,0.030384,0.984923,0.015250
2,0.75,0.030384,0.977469,0.010167
3,1.00,0.030384,0.970073,0.007625
4,1.25,0.030510,0.962580,0.006227
...,...,...,...,...
115,29.00,0.040499,0.308986,0.000361
116,29.25,0.040509,0.305779,0.000359
117,29.50,0.040520,0.302603,0.000356
118,29.75,0.040531,0.299456,0.000353


### Pairwise Forward Rates Between Successive Tenors
Use adjacent discount factors to calculate forward rates over each sub-period.


In [4]:
tenors = discount_curve_df["tenor_years"].values
fwd_rows = []
for t1, t2 in zip(tenors[:-1], tenors[1:]):
    df1 = discount_curve.discount_factor(t1)
    df2 = discount_curve.discount_factor(t2)
    forward = (df1 / df2 - 1.0) / (t2 - t1)
    fwd_rows.append({"T1": t1, "T2": t2, "forward_rate": forward})
forward_from_discount = pd.DataFrame(fwd_rows)
forward_from_discount


,T1,T2,forward_rate
0,0.25,0.50,0.030500
1,0.50,0.75,0.030500
2,0.75,1.00,0.030500
3,1.00,1.25,0.031135
4,1.25,1.50,0.031390
...,...,...,...
114,28.75,29.00,0.041906
115,29.00,29.25,0.041949
116,29.25,29.50,0.041993
117,29.50,29.75,0.042038


### Inspect Forward Zero Curve Inputs
Display the SONIA swap quote dataframe backing the forward projection curve.


In [5]:
forward_df


,instrument_type,tenor_years,rate
0,SONIA_SWAP,1,0.036002
1,SONIA_SWAP,2,0.034944
2,SONIA_SWAP,3,0.034997
3,SONIA_SWAP,4,0.035321
4,SONIA_SWAP,5,0.035766
5,SONIA_SWAP,6,0.036316
6,SONIA_SWAP,7,0.036944
7,SONIA_SWAP,8,0.037611
8,SONIA_SWAP,9,0.038285
9,SONIA_SWAP,10,0.038969


### Visualise Discount and Forward Zero Curves
Plot SONIA discount and forward zero-rate term structures for comparison.


In [6]:
discount_curve_df = discount_curve.as_dataframe()
forward_curve_df = forward_curve.as_dataframe().rename(columns={"zero_rate": "forward_zero_rate", "discount_factor": "forward_discount"})

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=discount_curve_df["tenor_years"],
        y=discount_curve_df["zero_rate"],
        mode="lines+markers",
        name="Discount Zero Rates",
    )
)
fig.add_trace(
    go.Scatter(
        x=forward_curve_df["tenor_years"],
        y=forward_curve_df["forward_zero_rate"],
        mode="lines+markers",
        name="Forward Zero Rates",
    )
)
fig.update_layout(
    title="SONIA Discount vs Forward Zero Curves",
    xaxis_title="Tenor (years)",
    yaxis_title="Zero Rate",
    template="plotly_white",
)
fig


## Swap Assumptions


### Define Swap Trade Parameters
Set valuation date, effective date, notional (GBP 10mm), fixed rate (current 5Y SONIA mid-swap), payment frequencies, and day-count conventions for the GBP SONIA swap.


In [7]:
swap_definition = SwapDefinition(
    valuation_date=date(2025, 11, 13),
    effective_date=date(2025, 11, 17),
    maturity_years=5,
    notional=10_000_000,
    fixed_rate=0.035766,
    payer="fixed",
    fixed_leg_frequency=2,
    floating_leg_frequency=4,
    fixed_leg_daycount="30/360",
    floating_leg_daycount="ACT/365",
    spread=0.0,
)
swap_definition


SwapDefinition(valuation_date=datetime.date(2025, 11, 13), effective_date=datetime.date(2025, 11, 17), maturity_years=5, notional=10000000, fixed_rate=0.035766, payer='fixed', fixed_leg_frequency=2, floating_leg_frequency=4, fixed_leg_daycount='30/360', floating_leg_daycount='ACT/365', spread=0.0)

### Build Cashflows and Initial Pricing Result
Instantiate the swap pricer, generate leg cashflows, and preview the first few rows.


In [8]:
pricer = SwapPricer(discount_curve=discount_curve, forward_curve=forward_curve)
pricing_result = pricer.price(swap_definition)

pricing_result["cashflows"].head()


,period_start,period_end,accrual_factor,coupon_rate,forward_rate,cashflow,discount_factor,present_value,time_to_payment,leg,projection_rate
0,2025-11-17,2026-02-17,0.252055,0.035841,0.035841,90338.826943,0.992040,89619.762242,0.263014,floating,0.035841
1,2025-11-17,2026-05-17,0.500000,0.035766,NaN,-178830.000000,0.984718,-176097.068570,0.506849,fixed,0.035766
2,2026-02-17,2026-05-17,0.243836,0.035841,0.035841,87392.995630,0.984718,86057.430766,0.506849,floating,0.035841
3,2026-05-17,2026-08-17,0.252055,0.035841,0.035841,90338.826943,0.977205,88279.557611,0.758904,floating,0.035841
4,2026-05-17,2026-11-17,0.500000,0.035766,NaN,-178830.000000,0.969743,-173419.141312,1.010959,fixed,0.035766


### Summarise Leg Present Values and NPV
Aggregate discounted cashflows by leg and compute overall mark-to-market value.


In [9]:
cashflows = pricing_result["cashflows"]

pv_summary = (
    cashflows.groupby("leg")["present_value"].sum().rename("present_value")
).to_frame()
pv_summary.loc["swap_npv", "present_value"] = pricing_result["npv"]

cashflow_display_cols = [
    "period_start",
    "period_end",
    "leg",
    "accrual_factor",
    "coupon_rate",
    "forward_rate",
    "cashflow",
    "discount_factor",
    "present_value",
]

cashflows_pretty = cashflows.copy()
cashflows_pretty["cashflow"] = cashflows_pretty["cashflow"].apply(lambda x: f"£{x:,.2f}")
cashflows_pretty["present_value"] = cashflows_pretty["present_value"].apply(lambda x: f"£{x:,.2f}")

pv_summary, cashflows_pretty[cashflow_display_cols]


(          present_value
 leg                    
 fixed     -1.638151e+06
 floating   1.638912e+06
 swap_npv   7.613190e+02,
    period_start  period_end       leg  accrual_factor  coupon_rate  \
 0    2025-11-17  2026-02-17  floating        0.252055     0.035841   
 1    2025-11-17  2026-05-17     fixed        0.500000     0.035766   
 2    2026-02-17  2026-05-17  floating        0.243836     0.035841   
 3    2026-05-17  2026-08-17  floating        0.252055     0.035841   
 4    2026-05-17  2026-11-17     fixed        0.500000     0.035766   
 5    2026-08-17  2026-11-17  floating        0.252055     0.035783   
 6    2026-11-17  2027-02-17  floating        0.252055     0.034479   
 7    2026-11-17  2027-05-17     fixed        0.500000     0.035766   
 8    2027-02-17  2027-05-17  floating        0.243836     0.033956   
 9    2027-05-17  2027-08-17  floating        0.252055     0.033414   
 10   2027-05-17  2027-11-17     fixed        0.500000     0.035766   
 11   2027-08-17  2027

### Detailed Cashflow Schedule
Display the full discounted cashflow table for both fixed and floating legs.


### Combined Cashflow View
Consolidate fixed and floating legs into a single table with discount factors, forward rates, and net present values per payment date.


In [10]:
combined_rows = []
for period_end, group in cashflows.groupby("period_end"):
    group = group.copy()
    discount_factor = group["discount_factor"].mean()
    time_to_payment = group["time_to_payment"].mean()

    fixed_cf = group.loc[group["leg"] == "fixed", "cashflow"].sum()
    floating_cf = group.loc[group["leg"] == "floating", "cashflow"].sum()

    fixed_rate = group.loc[group["leg"] == "fixed", "coupon_rate"].mean()
    floating_rate = group.loc[group["leg"] == "floating", "coupon_rate"].mean()
    forward_rate = group.loc[group["leg"] == "floating", "forward_rate"].mean()

    fixed_pv = group.loc[group["leg"] == "fixed", "present_value"].sum()
    floating_pv = group.loc[group["leg"] == "floating", "present_value"].sum()

    combined_rows.append(
        {
            "period_end": pd.to_datetime(period_end),
            "time_to_payment": time_to_payment,
            "discount_factor": discount_factor,
            "forward_rate": forward_rate,
            "fixed_coupon_rate": fixed_rate,
            "floating_coupon_rate": floating_rate,
            "fixed_cashflow": fixed_cf,
            "floating_cashflow": floating_cf,
            "net_cashflow": fixed_cf + floating_cf,
            "fixed_present_value": fixed_pv,
            "floating_present_value": floating_pv,
            "net_present_value": fixed_pv + floating_pv,
        }
    )

combined_cashflows = pd.DataFrame(combined_rows).sort_values("period_end").reset_index(drop=True)

cashflow_columns = [
    "fixed_cashflow",
    "floating_cashflow",
    "net_cashflow",
    "fixed_present_value",
    "floating_present_value",
    "net_present_value",
]

combined_cashflows_formatted = combined_cashflows.copy()
combined_cashflows_formatted[cashflow_columns] = (
    combined_cashflows_formatted[cashflow_columns].applymap(lambda x: f"£{x:,.2f}")
)
combined_cashflows_formatted


C:\Users\user\AppData\Local\Temp\ipykernel_25224\11925498.py:47: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,period_end,time_to_payment,discount_factor,forward_rate,fixed_coupon_rate,floating_coupon_rate,fixed_cashflow,floating_cashflow,net_cashflow,fixed_present_value,floating_present_value,net_present_value
0,2026-02-17,0.263014,0.992040,0.035841,NaN,0.035841,£0.00,"£90,338.83","£90,338.83",£0.00,"£89,619.76","£89,619.76"
1,2026-05-17,0.506849,0.984718,0.035841,0.035766,0.035841,"£-178,830.00","£87,393.00","£-91,437.00","£-176,097.07","£86,057.43","£-90,039.64"
2,2026-08-17,0.758904,0.977205,0.035841,NaN,0.035841,£0.00,"£90,338.83","£90,338.83",£0.00,"£88,279.56","£88,279.56"
3,2026-11-17,1.010959,0.969743,0.035783,0.035766,0.035783,"£-178,830.00","£90,192.59","£-88,637.41","£-173,419.14","£87,463.64","£-85,955.50"
4,2027-02-17,1.263014,0.962189,0.034479,NaN,0.034479,£0.00,"£86,905.73","£86,905.73",£0.00,"£83,619.71","£83,619.71"
5,2027-05-17,1.506849,0.954879,0.033956,0.035766,0.033956,"£-178,830.00","£82,795.81","£-96,034.19","£-170,761.03","£79,059.99","£-91,701.05"
6,2027-08-17,1.758904,0.947320,0.033414,NaN,0.033414,£0.00,"£84,220.46","£84,220.46",£0.00,"£79,783.76","£79,783.76"
7,2027-11-17,2.010959,0.939754,0.032980,0.035766,0.032980,"£-178,830.00","£83,128.59","£-95,701.41","£-168,056.13","£78,120.38","£-89,935.74"
8,2028-02-17,2.263014,0.932029,0.034916,NaN,0.034916,£0.00,"£88,007.51","£88,007.51",£0.00,"£82,025.57","£82,025.57"
9,2028-05-17,2.509589,0.924441,0.034943,0.035766,0.034943,"£-178,830.00","£86,160.88","£-92,669.12","£-165,317.85","£79,650.68","£-85,667.17"


### Swap Risk Measures
Compute PV01/DV01 style sensitivities alongside the mark-to-market NPV.


In [11]:
try:
    ZeroCurve
    CurvePoint
except NameError:
    from src.curves import ZeroCurve, CurvePoint

import numpy as np

def bump_curve(curve: ZeroCurve, bump_bp: float) -> ZeroCurve:
    bump = bump_bp / 10_000.0
    tenors = curve.tenors
    base_dfs = np.array([curve.discount_factor(t) for t in tenors])
    bumped_dfs = base_dfs * np.exp(-bump * tenors)
    bumped_rates = -np.log(bumped_dfs) / tenors
    bumped_points = [CurvePoint(t, r) for t, r in zip(tenors, bumped_rates)]
    return ZeroCurve(bumped_points, name=f"{curve.name} bump {bump_bp}bp", discount_factors=bumped_dfs)

bump_size_bp = 1.0
bumped_curve = bump_curve(discount_curve, bump_size_bp)

bumped_pricer = SwapPricer(discount_curve=bumped_curve, forward_curve=forward_curve)
pricing_bumped = bumped_pricer.price(swap_definition)

npv = pricing_result["npv"]
npv_bumped = pricing_bumped["npv"]
pv01 = npv_bumped - npv
dv01 = -pv01

risk_summary = pd.DataFrame(
    {
        "metric": ["NPV", "PV01", "DV01"],
        "value": [npv, pv01, dv01],
    }
)

risk_summary_formatted = risk_summary.copy()
risk_summary_formatted["value"] = risk_summary_formatted["value"].apply(lambda x: f"£{x:,.2f}")
risk_summary_formatted


,metric,value
0,NPV,£761.32
1,PV01,£14.73
2,DV01,£-14.73


### 50bp Parallel SONIA Curve Stress
Apply a +50 basis-point parallel shift to both the SONIA discount and forward curves and compare against the base curves.


In [12]:
shift_bp = 50.0
shift = shift_bp / 10_000.0

stress_tenors = discount_curve.tenors
base_zero_rates = discount_curve.zero_rates
stressed_zero_rates = base_zero_rates + shift
stressed_discount_factors = np.exp(-stressed_zero_rates * stress_tenors)

stressed_discount_curve = ZeroCurve(
    [CurvePoint(t, r) for t, r in zip(stress_tenors, stressed_zero_rates)],
    name=f"{discount_curve.name} +{shift_bp:.0f}bp",
    discount_factors=stressed_discount_factors,
)

forward_stress_tenors = forward_curve.tenors
forward_base_rates = forward_curve.zero_rates
forward_stressed_rates = forward_base_rates + shift
forward_stressed_dfs = np.exp(-forward_stressed_rates * forward_stress_tenors)

stressed_forward_curve = ZeroCurve(
    [CurvePoint(t, r) for t, r in zip(forward_stress_tenors, forward_stressed_rates)],
    name=f"{forward_curve.name} +{shift_bp:.0f}bp",
    discount_factors=forward_stressed_dfs,
)

stress_curve_df = pd.DataFrame(
    {
        "tenor_years": stress_tenors,
        "base_zero_rate": base_zero_rates,
        "stressed_zero_rate": stressed_zero_rates,
        "forward_base_zero_rate": forward_curve.zero_rates,
        "forward_stressed_zero_rate": forward_stressed_rates,
    }
)

stress_fig = go.Figure()
stress_fig.add_trace(
    go.Scatter(
        x=stress_curve_df["tenor_years"],
        y=stress_curve_df["base_zero_rate"],
        mode="lines+markers",
        name="Base Discount Curve",
    )
)
stress_fig.add_trace(
    go.Scatter(
        x=stress_curve_df["tenor_years"],
        y=stress_curve_df["stressed_zero_rate"],
        mode="lines+markers",
        name="Stressed Discount Curve (+50bp)",
    )
)
stress_fig.add_trace(
    go.Scatter(
        x=forward_stress_tenors,
        y=forward_base_rates,
        mode="lines+markers",
        name="Base Forward Curve",
    )
)
stress_fig.add_trace(
    go.Scatter(
        x=forward_stress_tenors,
        y=forward_stressed_rates,
        mode="lines+markers",
        name="Stressed Forward Curve (+50bp)",
    )
)
stress_fig.update_layout(
    title="SONIA Discount & Forward Curves Before and After 50bp Shift",
    xaxis_title="Tenor (years)",
    yaxis_title="Zero Rate",
    template="plotly_white",
)
stress_fig


### Stressed Swap Valuation (+50bp)
Compute the swap mark-to-market and risk measures under the stressed discount curve.


In [13]:
stressed_pricer = SwapPricer(discount_curve=stressed_discount_curve, forward_curve=stressed_forward_curve)
stressed_pricing_result = stressed_pricer.price(swap_definition)
stressed_npv = stressed_pricing_result["npv"]

bumped_stressed_curve = bump_curve(stressed_discount_curve, 1.0)
bumped_stressed_pricing = SwapPricer(
    discount_curve=bumped_stressed_curve,
    forward_curve=stressed_forward_curve,
).price(swap_definition)
stressed_pv01 = bumped_stressed_pricing["npv"] - stressed_npv
stressed_dv01 = -stressed_pv01

stressed_summary = pd.DataFrame(
    {
        "metric": [
            "Stressed NPV (+50bp)",
            "Stressed PV01 (+50bp)",
            "Stressed DV01 (+50bp)",
        ],
        "value": [stressed_npv, stressed_pv01, stressed_dv01],
    }
)
stressed_summary_formatted = stressed_summary.copy()
stressed_summary_formatted["value"] = stressed_summary_formatted["value"].apply(lambda x: f"£{x:,.2f}")
stressed_summary_formatted


,metric,value
0,Stressed NPV (+50bp),"£228,645.17"
1,Stressed PV01 (+50bp),£-43.69
2,Stressed DV01 (+50bp),£43.69


### Trade Summary
Key economic details of the SONIA swap trade.


In [14]:
npv_value = pricing_result["npv"]
pv01_value = pv01
dv01_value = dv01
fair_rate = forward_curve.forward_rate(0, swap_definition.maturity_years) * 100

summary_table = pd.DataFrame(
    {
        "Attribute": [
            "Notional",
            "Currency",
            "Fixed Rate",
            "Swap Type",
            "Valuation Date",
            "Effective Date",
            "Maturity",
            "Fixed Leg Frequency",
            "Floating Leg Frequency",
            "Discount Curve",
            "Forward Curve",
            "Fair Swap Rate",
            "Mark-to-Market",
            "PV01",
            "DV01",
            "Stressed NPV (+50bp)",
            "Stressed PV01 (+50bp)",
            "Stressed DV01 (+50bp)",
        ],
        "Value": [
            f"£{swap_definition.notional:,.0f}",
            "GBP",
            f"{swap_definition.fixed_rate*100:.4f} %",
            "Fixed Payer" if swap_definition.payer == "fixed" else "Fixed Receiver",
            swap_definition.valuation_date.isoformat(),
            swap_definition.effective_date.isoformat(),
            f"{swap_definition.maturity_years:.1f} years",
            f"{swap_definition.fixed_leg_frequency} per year",
            f"{swap_definition.floating_leg_frequency} per year",
            discount_curve.name,
            forward_curve.name,
            f"{fair_rate:.4f} %",
            f"£{npv_value:,.2f}",
            f"£{pv01_value:,.2f}",
            f"£{dv01_value:,.2f}",
            f"£{stressed_npv:,.2f}",
            f"£{stressed_pv01:,.2f}",
            f"£{stressed_dv01:,.2f}",
        ],
    }
)
summary_table


,Attribute,Value
0,Notional,"£10,000,000"
1,Currency,GBP
2,Fixed Rate,3.5766 %
3,Swap Type,Fixed Payer
4,Valuation Date,2025-11-13
5,Effective Date,2025-11-17
6,Maturity,5.0 years
7,Fixed Leg Frequency,2 per year
8,Floating Leg Frequency,4 per year
9,Discount Curve,SONIA OIS Discount


In [15]:
cashflows


,period_start,period_end,accrual_factor,coupon_rate,forward_rate,cashflow,discount_factor,present_value,time_to_payment,leg,projection_rate
0,2025-11-17,2026-02-17,0.252055,0.035841,0.035841,90338.826943,0.992040,89619.762242,0.263014,floating,0.035841
1,2025-11-17,2026-05-17,0.500000,0.035766,NaN,-178830.000000,0.984718,-176097.068570,0.506849,fixed,0.035766
2,2026-02-17,2026-05-17,0.243836,0.035841,0.035841,87392.995630,0.984718,86057.430766,0.506849,floating,0.035841
3,2026-05-17,2026-08-17,0.252055,0.035841,0.035841,90338.826943,0.977205,88279.557611,0.758904,floating,0.035841
4,2026-05-17,2026-11-17,0.500000,0.035766,NaN,-178830.000000,0.969743,-173419.141312,1.010959,fixed,0.035766
5,2026-08-17,2026-11-17,0.252055,0.035783,0.035783,90192.594048,0.969743,87463.637044,1.010959,floating,0.035783
6,2026-11-17,2027-02-17,0.252055,0.034479,0.034479,86905.733255,0.962189,83619.710442,1.263014,floating,0.034479
7,2026-11-17,2027-05-17,0.500000,0.035766,NaN,-178830.000000,0.954879,-170761.032571,1.506849,fixed,0.035766
8,2027-02-17,2027-05-17,0.243836,0.033956,0.033956,82795.807003,0.954879,79059.987119,1.506849,floating,0.033956
9,2027-05-17,2027-08-17,0.252055,0.033414,0.033414,84220.461303,0.947320,79783.764900,1.758904,floating,0.033414


### Forward Rate Projection by Reset Period
Generate a table of quarterly SONIA forward rates implied by the forward curve.


In [16]:
forward_tenors = np.linspace(0.25, swap_definition.maturity_years, int(swap_definition.maturity_years * swap_definition.floating_leg_frequency))
forward_rates = [forward_curve.forward_rate(t - 0.25, t) for t in forward_tenors]

forward_projection = pd.DataFrame({
    "period_end_years": forward_tenors,
    "forward_rate": forward_rates,
})
forward_projection


,period_end_years,forward_rate
0,0.25,0.035841
1,0.50,0.035841
2,0.75,0.035841
3,1.00,0.035841
4,1.25,0.034507
5,1.50,0.033971
6,1.75,0.033433
7,2.00,0.032892
8,2.25,0.034915
9,2.50,0.034942


### Visualise Forward SONIA Path
Plot the quarterly forward rates to highlight expectations over the swap horizon.


In [17]:
fig_forward = go.Figure()
fig_forward.add_trace(
    go.Bar(
        x=forward_projection["period_end_years"],
        y=forward_projection["forward_rate"],
        name="Forward SONIA",
    )
)
fig_forward.update_layout(
    title="Forward SONIA Rates by Period",
    xaxis_title="Time (years)",
    yaxis_title="Forward Rate",
    template="plotly_white",
)
fig_forward
